# Data Modeling I: Probability and Statistics

## Introduction

### Why Probability and Statistics Matter in Science

Any physical measurement involves some level of uncertainty or noise.
Whether we are measuring the intensity of a star's light, the decay rate of a radioactive sample, or the temperature of the cosmic microwave background, the data we collect are never perfectly exact.
Probability theory offers a systematic way to handle this lack of certainty.
More specifically:
* **Data Interpretation**:
  We want to connect noisy observations to underlying physical models.
  If a dataset appears to fluctuate, is that signal real, or is it random?
  Probability gives us formal tools—like hypothesis testing or confidence intervals—to decide.
* **Incomplete Knowledge**:
  Even when processes are entirely deterministic at some level, we often lack complete information.
  Probability distributions let us quantify the range of possible outcomes or parameter values.
* **Unrepeatable Events**:
  Fields like astronomy pose a unique challenge: many phenomena (e.g., a supernova) cannot be restarted under controlled conditions.
  We must rely on "fair samples" of data from one-time observations.
  Probability theory becomes crucial for making sense of these non-repeatable experiments.

From a broad perspective, probability theory is the logic of science:
it extends our classical (Boolean) logic into a realm where conclusions cannot be absolutely certain but can be assigned degrees of belief or confidence.

### Historical Context and Key Contributors

Probability theory and its practical offshoot—statistics—did not emerge fully formed.
Many scientists who pioneered the subject were themselves astronomers or physicists grappling with noisy measurements:
* **Blaise Pascal & Pierre Fermat** (1650s):
  Their work on games of chance launched the formal study of probability, initially focusing on gambling problems but laying the groundwork for more general applications.
* **Jacob Bernoulli & Thomas Bayes** (18th century):
  They introduced foundational ideas on how to assign and update probabilities.
  Bayes's Theorem still underlies modern Bayesian statistics, which treats probability as "degree of belief" and updates those degrees using observed data.
* **Pierre-Simon Laplace & Carl Friedrich Gauss** (late 18th/early 19th century):
  Both were astronomers/mathematicians.
  Gauss's work on least squares and the "Gaussian (normal) distribution of errors" became central to how we handle measurement noise.
  Laplace's rediscovery of Bayesian methods brought probability firmly into the domain of scientific data interpretation.
* **Frequentist vs. Bayesian** (20th century):
  Mathematicians and statisticians debated how best to define, interpret, and use probabilities, especially for inference.
  The result was a rich theoretical framework that scientists still apply daily.